In [112]:
import os
import json
from tqdm.notebook import tqdm
import re
import uuid
import random
import pandas as pd

In [2]:
os.listdir('../data/sohu')

['8岁女孩下体受伤案，究竟有无老师唆使？.txt',
 '深圳GDP首次超越香港，成粤港澳大湾区经济总量第一城.txt',
 '拆除“削山别墅”，更得打击违建者嚣张气焰.txt',
 '“投币大军”再增一员这“病”如何治.txt',
 '山西太原明星市长耿彦波卸任，曾被称现实版李达康.txt',
 '权健公司束某某等16人被依法批准逮捕.txt',
 '起底城市绿化中的奢靡之风：贫困县斥资200多万种银杏.txt',
 '从警35年的副市长退休三年被挖出，曾被举报向黑老大赠送字画.txt',
 '百名婴幼儿口服过期疫苗，责任心去哪了？.txt',
 '陕西省委原书记赵正永涉严重违纪违法接受纪律审查和监察调查.txt',
 '呼和浩特发布大学毕业生“半价购房”新政，专家：力度大或作用有限.txt',
 '辽宁教育厅回应“等8年突然被取消高考资格”：改革需要.txt',
 '未婚女孩6年做17次流产，子宫薄如纸！医生都看不下去了…….txt',
 '龙永图：多向美国进口是一种进步 符合贸易政策调整方向.txt',
 '加总理称中国随意作死刑判决，中国外交部：加方缺乏法治精神.txt',
 '捷克总理：华为威胁国家安全？没证据.txt']

In [3]:
file_path_list = ['../data/sohu/' + item for item in os.listdir('../data/sohu')]
file_path_list

['../data/sohu/8岁女孩下体受伤案，究竟有无老师唆使？.txt',
 '../data/sohu/深圳GDP首次超越香港，成粤港澳大湾区经济总量第一城.txt',
 '../data/sohu/拆除“削山别墅”，更得打击违建者嚣张气焰.txt',
 '../data/sohu/“投币大军”再增一员这“病”如何治.txt',
 '../data/sohu/山西太原明星市长耿彦波卸任，曾被称现实版李达康.txt',
 '../data/sohu/权健公司束某某等16人被依法批准逮捕.txt',
 '../data/sohu/起底城市绿化中的奢靡之风：贫困县斥资200多万种银杏.txt',
 '../data/sohu/从警35年的副市长退休三年被挖出，曾被举报向黑老大赠送字画.txt',
 '../data/sohu/百名婴幼儿口服过期疫苗，责任心去哪了？.txt',
 '../data/sohu/陕西省委原书记赵正永涉严重违纪违法接受纪律审查和监察调查.txt',
 '../data/sohu/呼和浩特发布大学毕业生“半价购房”新政，专家：力度大或作用有限.txt',
 '../data/sohu/辽宁教育厅回应“等8年突然被取消高考资格”：改革需要.txt',
 '../data/sohu/未婚女孩6年做17次流产，子宫薄如纸！医生都看不下去了…….txt',
 '../data/sohu/龙永图：多向美国进口是一种进步 符合贸易政策调整方向.txt',
 '../data/sohu/加总理称中国随意作死刑判决，中国外交部：加方缺乏法治精神.txt',
 '../data/sohu/捷克总理：华为威胁国家安全？没证据.txt']

In [144]:
# 统计搜狐评论数量
comment_num = 0
sohu_comment_list = []
for item in file_path_list:
    with open(item, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            sample = {
                    'uid': str(uuid.uuid1()),
                    'comment': line,
                    'line': line,
            }
            sohu_comment_list.append(sample)
        comment_num += len(lines)
print(comment_num)
print(len(sohu_comment_list))

32959
32959


In [145]:
# with open('../data/sohu_test_v0.json', 'w', encoding='utf-8') as f:
#     for item in sohu_comment_list:
#         write_str = json.dumps(item, ensure_ascii=False)
#         f.write(write_str)
#         f.write('\n')

In [147]:
sohu_comment_list[129]

{'uid': '0fd9d1b6-04a5-11ed-95f9-a683e78cc882',
 'comment': '看人家没有能出头的人，就这么欺负，简直太黑暗了，跟红楼梦里薛蟠打死人的情节太像了，堂堂法治社会，怎么能出这么黑暗的事！',
 'line': '看人家没有能出头的人，就这么欺负，简直太黑暗了，跟红楼梦里薛蟠打死人的情节太像了，堂堂法治社会，怎么能出这么黑暗的事！'}

In [5]:
# 统计微博评论数量
pos_file_path = '../data/pos60000.txt'
neg_file_path = '../data/neg60000.txt'

weibo_path_list = [pos_file_path, neg_file_path]

comment_num = 0
for item in weibo_path_list:
    with open(item, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        comment_num += len(lines)
print(comment_num)

120000


### 以搜狐评论不带标签数据 32959 为测试集
### 以微博评论具有标签数据 120000 位训练集 + 补充开源的中文情绪分类数据
### 微博评论数据存在问题
* 怀疑训练数据的标签是根据 emoji 表情自动标准的，标签准确率需要评估
* 带有很多转载，\@ 符合，用户昵称本身等，对整个评论情绪识别干扰，需要过滤处理

In [101]:
weibo_comment_list = []
zhuan_zai_reg = re.compile(r'\//[\s]*@.*?[\:\：]{1}')
for item in weibo_path_list:
    with open(item, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in tqdm(lines):
            line = line.strip()
            comment_list = re.split(zhuan_zai_reg, line)
            if item == pos_file_path:
                sample = {
                    'uid': str(uuid.uuid1()),
                    'comment': ''.join(comment_list).lstrip().rstrip(),
                    'line': line,
                    'sentiment_label': 1
                }
                weibo_comment_list.append(sample)
            else:
                sample = {
                    'uid': str(uuid.uuid1()),
                    'comment': ''.join(comment_list).lstrip().rstrip(),
                    'line': line,
                    'sentiment_label': 0
                }
                weibo_comment_list.append(sample)
                    
len(weibo_comment_list)

120000

In [142]:
sample_index = random.randint(0, 119999)
random.shuffle(weibo_comment_list)
weibo_comment_list[sample_index]

{'uid': '92a34ad2-044b-11ed-95f9-a683e78cc882',
 'comment': '哈，下次我赶着一头驴，唱着“我的故乡在远方”去旅行。[爱你]',
 'line': '哈，下次我赶着一头驴，唱着“我的故乡在远方”去旅行。[爱你]',
 'sentiment_label': 1}

In [109]:
# 随机抽样 100 条，得到训练集精度

weibo_comment_list_demo = random.sample(weibo_comment_list, 100)
len(weibo_comment_list_demo), weibo_comment_list_demo[0]

(100,
 {'uid': '92b9f930-044b-11ed-95f9-a683e78cc882',
  'comment': '[偷笑],办法满好的，就是接着只好趴着睡觉，枕头上要挖个洞[哈哈]',
  'line': '[偷笑],办法满好的，就是接着只好趴着睡觉，枕头上要挖个洞[哈哈]',
  'sentiment_label': 1})

In [113]:
# 将处理好的训练集写成 json 文件
# 将抽样数据 100 条整成 excel

# with open('../data/weibo_train_v0_by_emoji.json', 'w', encoding='utf-8') as f:
#     for item in weibo_comment_list:
#         write_str = json.dumps(item, ensure_ascii=False)
#         f.write(write_str)
#         f.write('\n')

# new_df = pd.DataFrame()
# new_df['uid'] = [item['uid'] for item in weibo_comment_list_demo]
# new_df['comment'] = [item['comment'] for item in weibo_comment_list_demo]
# new_df['line'] = [item['line'] for item in weibo_comment_list_demo]
# new_df['sentiment_label'] = [item['sentiment_label'] for item in weibo_comment_list_demo]
# print(new_df.shape)
# new_df.to_excel('../data/weibo_train_v0_by_emoji_sample_100.xlsx', index=False)

(100, 4)


In [ ]:
# 经过人工验证 sentiment_label ，标签准确率为 93%
# 可以搞第一版模型了
# baseline 模型 —— bert